$$\textbf{Chad Schupbach}$$

***

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from src import utils

Using TensorFlow backend.


A common CNN feature learning architecture is similar to that shown above, where feature maps are constructed with ReLU-activated convolutional layers and intermittent max pooling. It's also fairly common to normalize the batch of input layers prior to convolution in the feature learning segment.

## Classification

In the classification segment of the network, we flatten the final laywe shift our focus to classification using fully connected (FC) layers.

### Fully Connected (FC) Layers

$$\begin{aligned} \underset{28 \times 28 \times 1}{ \ \ \textsf{input}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{28 \times 28 \times 16}{ \ \ \textsf{conv1}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{28 \times 28 \times 16}{\textsf{conv2}^\dagger} \ \ _{ \ast \ \ \text{max pool}_{2 \times 2} \ \ \longmapsto}\\[5pt]
\underset{14 \times 14 \times 16}{ \, \ \textsf{pool1}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{14 \times 14 \times 32}{\textsf{conv3}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{14 \times 14 \times 32}{\textsf{conv4}^\dagger} \ \ _{ \ast \ \ \text{max pool}_{2 \times 2} \ \ \longmapsto}\\[5pt]
\underset{7 \times 7 \times 32}{ \, \ \textsf{pool2}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{7 \times 7 \times 64}{ \, \ \textsf{conv5}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{7 \times 7 \times 64}{ \, \ \textsf{conv6}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto}\\[5pt] \underset{7 \times 7 \times 64}{ \, \ \textsf{conv7}^\dagger} \ \ _{ + \ \ \text{flatten}_{1600} \ + \ \text{dropout}_{0.25} \ + \ \text{dense}_{128} \ + \ \text{ReLU} \ \ \longmapsto}\\[10pt]
\underset{1 \times 1 \times 128}{\textsf{FC6}} \ \ _{ + \ \ \text{dropout}_{0.5} \ + \ \text{dense}_{128} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{1 \times 1 \times 128}{\textsf{FC7}} \ \ _{ + \ \ \text{dropout}_{0.5} \ + \ \text{dense}_{10} \ + \ \text{Softmax} \ \ \longmapsto} \ \ \underset{1 \times 1 \times 10}{\textsf{output}} \end{aligned}$$

# Initialization

Load the entire MNIST dataset containing 60000 training images and 10000 testing images across 10 classes $\{0,1,\dots,8,9\}$.

In [2]:
x_train, y_train, x_test, y_test, input_shape = utils.load_mnist(method='keras')

For the ensemble, we will train 5 models with the same architecture using a batch size of 128.

In [3]:
n_classes = 10
batch_size = 128
n_models = 10

The architecture of each CNN model is as follows:

In [4]:
model = [None] * n_models
for i in range(n_models):
    model[i] = Sequential()
    model[i].add(Conv2D(16, 3, padding='same', activation='relu',
                        input_shape=(28, 28, 1)))
    model[i].add(Conv2D(16, 3, padding='same', activation='relu'))
    model[i].add(MaxPooling2D(pool_size=(2, 2)))
    model[i].add(Conv2D(32, 3, padding='same', activation='relu'))
    model[i].add(Conv2D(32, 3, padding='same', activation='relu'))
    model[i].add(MaxPooling2D(pool_size=(2, 2)))
    model[i].add(Conv2D(64, 3, padding='same', activation='relu'))
    model[i].add(Conv2D(64, 3, padding='same', activation='relu'))
    model[i].add(Conv2D(64, 3, activation='relu'))
    model[i].add(Flatten())
    model[i].add(Dropout(0.25))
    model[i].add(Dense(batch_size, activation='relu'))
    model[i].add(Dropout(0.5))
    model[i].add(Dense(batch_size, activation='relu'))
    model[i].add(Dropout(0.5))
    model[i].add(Dense(n_classes, activation='softmax'))
    model[i].compile(optimizer='nadam', loss='categorical_crossentropy',
                     metrics=['accuracy'])

In [5]:
model[0].summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)         

We assign model checkpoints and early stopping criterion below. The checkpoints save parameter weights of the best training epoch based on validation accuracy to files in the `models` directory. We also set early stopping criterion that indicates convergence when no decrease in validation loss is observed over 8 epochs.

In [6]:
checkpoint = []
earlystop = []
for i in range(n_models):
    checkpoint += [ModelCheckpoint(filepath=f'models/best_weights_{i+1}.hdf5',
                                   monitor='val_accuracy', save_best_only=True,
                                   save_weights_only=True, mode='max')]
    earlystop += [EarlyStopping(monitor='val_loss', patience=8, mode='min')]

If the early stopping criterion is not met, we end the training session of each model after 25 epochs. Because we need a validation set for the checkpoints and convergence criterion, we randomly split the training data into 80% training and 20% validation prior to each training session. It's not uncommon to see similar models implemented using the testing set for model validation. While it's tempting given that the validation set is not actually being used in training, this strategy will result in model selection bias and should be avoided.

In [7]:
epochs = 25
ledger = []
for i in range(n_models):
    clear_output(wait=True)
    print(f'CNN_{i+1}\n' + '='*50)
    x_train2, x_valid, y_train2, y_valid = train_test_split(x_train, y_train,
                                                            test_size=0.2)
    ledger += [model[i].fit(x_train2, y_train2, batch_size=batch_size, epochs=epochs,
                            verbose=1, callbacks=[checkpoint[i], earlystop[i]],
                            validation_data=(x_valid, y_valid))]

CNN_10
Train on 48000 samples, validate on 12000 samples
Epoch 1/25
48000/48000 [==============================] - 47s 973us/step - loss: 0.4115 - accuracy: 0.8711 - val_loss: 0.0706 - val_accuracy: 0.9822
Epoch 2/25
48000/48000 [==============================] - 44s 925us/step - loss: 0.0946 - accuracy: 0.9771 - val_loss: 0.0534 - val_accuracy: 0.9859
Epoch 3/25
48000/48000 [==============================] - 44s 926us/step - loss: 0.0685 - accuracy: 0.9829 - val_loss: 0.0441 - val_accuracy: 0.9887
Epoch 4/25
48000/48000 [==============================] - 44s 926us/step - loss: 0.0560 - accuracy: 0.9864 - val_loss: 0.0492 - val_accuracy: 0.9866
Epoch 5/25
48000/48000 [==============================] - 44s 925us/step - loss: 0.0527 - accuracy: 0.9867 - val_loss: 0.0386 - val_accuracy: 0.9893
Epoch 6/25
48000/48000 [==============================] - 45s 929us/step - loss: 0.0402 - accuracy: 0.9907 - val_loss: 0.0381 - val_accuracy: 0.9907
Epoch 7/25
48000/48000 [=========================

In [8]:
opt_epoch = []
train_loss = []
train_acc = []
val_loss = []
val_acc = []
for i in range(n_models):
    opt_epoch += [np.argmax(ledger[i].history['val_accuracy']) + 1]
    train_loss += [np.min(ledger[i].history['loss'])]
    train_acc += [np.max(ledger[i].history['accuracy'])]
    val_loss += [np.min(ledger[i].history['val_loss'])]
    val_acc += [np.max(ledger[i].history['val_accuracy'])]

In [9]:
test_loss = []
test_acc = []
for i in range(n_models):
    model[i].load_weights(f'models/best_weights_{i+1}.hdf5')
    score = model[i].evaluate(x_test, y_test, verbose=0)
    test_loss += [score[0]]
    test_acc += [score[1]]

In [10]:
df = pd.DataFrame({
    'Opt Epoch': opt_epoch + [np.mean(opt_epoch)],
    'Train Loss': train_loss + [np.mean(train_loss)],
    'Valid Loss': val_loss + [np.mean(val_loss)],
    'Test Loss': test_loss + [np.mean(test_loss)],
    'Train Acc': train_acc + [np.mean(train_acc)],
    'Valid Acc': val_acc + [np.mean(val_acc)],
    'Test Acc': test_acc + [np.mean(test_acc)]
}, index=[f'Model {i+1}' for i in range(n_models)] + ['Average']).round(5).round({
    'Train Acc': 4,
    'Valid Acc': 4,
    'Test Acc': 4
})

In [11]:
df

,Opt Epoch,Train Loss,Valid Loss,Test Loss,Train Acc,Valid Acc,Test Acc
Model 1,8.0,0.02268,0.03259,0.03060,0.9948,0.9920,0.9927
Model 2,11.0,0.02162,0.02976,0.03487,0.9946,0.9930,0.9925
Model 3,17.0,0.02584,0.04130,0.03551,0.9935,0.9917,0.9928
Model 4,11.0,0.02234,0.04165,0.03103,0.9944,0.9916,0.9924
Model 5,9.0,0.02704,0.03801,0.02933,0.9926,0.9912,0.9933
Model 6,20.0,0.02132,0.03396,0.03077,0.9950,0.9923,0.9924
Model 7,18.0,0.02626,0.03635,0.03011,0.9935,0.9921,0.9929
Model 8,13.0,0.02871,0.04469,0.03480,0.9930,0.9908,0.9931
Model 9,9.0,0.02939,0.03729,0.04206,0.9926,0.9911,0.9920
Model 10,15.0,0.02196,0.03338,0.03078,0.9949,0.9928,0.9927


In [12]:
train_res = np.zeros(y_train.shape)
test_res = np.zeros(y_test.shape)
for i in range(n_models):
    train_res += model[i].predict(x_train)
    test_res += model[i].predict(x_test)
train_res /= n_models
test_res /= n_models

In [13]:
train_pred = np.argmax(train_res, axis=1)
train_act = np.argmax(y_train, axis=1)
ens_train_acc = (train_pred == train_act).mean()
test_pred = np.argmax(test_res, axis=1)
test_act = np.argmax(y_test, axis=1)
ens_test_acc = (test_pred == test_act).mean()
print(f'Ensemble Train Accuracy: {ens_train_acc}')
print(f'Ensemble Test Accuracy: {ens_test_acc}')

Ensemble Train Accuracy: 0.99895
Ensemble Test Accuracy: 0.9961


In [30]:
# # df = pd.DataFrame()
# pd.DataFrame(train_res).to_csv('data/train_res.csv', index=False)
# pd.DataFrame(test_res).to_csv('data/test_res.csv', index=False)
# x1 = pd.read_csv('data/train_res.csv')
x1.to_numpy()[(train_pred != train_act),[train_pred[(train_pred != train_act)]]]
x1.to_numpy()[(train_pred != train_act),[train_act[(train_pred != train_act)]]]


array([9, 6, 8, 4, 9, 1, 1, 2, 3, 8, 3, 8, 9, 8, 9, 1, 3, 8, 7, 1, 1, 4,
       9, 5, 2, 7, 6, 4, 9, 9, 4, 9, 9, 4, 8, 5, 9, 6, 6, 1, 2, 1, 1, 9,
       0, 5, 2, 8, 5, 9, 7, 1, 6, 9, 7, 6, 9, 4, 8, 9, 5, 4, 4])

<module 'tensorflow' from '/Users/chadschupbach/opt/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py'>

In [3]:
model = Sequential()
model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)         

In [ ]:
# fp = 'models/best_weights.hdf5'
# checkpoint = ModelCheckpoint(fp, monitor='val_accuracy', save_best_only=True,
#                              save_weights_only=True, mode='max')

In [8]:
model.compile(optimizer='nadam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [9]:
model.fit(x_train, y_train, batch_size=128, epochs=2, verbose=1,
          validation_split=0.2, initial_epoch=0)

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 56s 1ms/sample - loss: 0.4966 - accuracy: 0.8381 - val_loss: 0.0673 - val_accuracy: 0.9805
Epoch 2/2
48000/48000 [==============================] - 52s 1ms/sample - loss: 0.1167 - accuracy: 0.9694 - val_loss: 0.0479 - val_accuracy: 0.9866


In [15]:
import numpy as np

arr = np.array([
    [0.4583, 0.8572, 0.0730, 0.9807],
    [0.1046, 0.9735, 0.0512, 0.9858],
    [0.0736, 0.9824, 0.0437, 0.9881],
    [0.0604, 0.9842, 0.0519, 0.9869],
    [0.0525, 0.9870, 0.0328, 0.9908],
    [0.0448, 0.9886, 0.0461, 0.9906],
    [0.0451, 0.9883, 0.0405, 0.9900],
    [0.0397, 0.9902, 0.0375, 0.9908],
    [0.0398, 0.9899, 0.0419, 0.9915],
    [0.0385, 0.9900, 0.0642, 0.9860],
    [0.0395, 0.9901, 0.0477, 0.9908],
    [0.0298, 0.9924, 0.0591, 0.9895],
    [0.0341, 0.9912, 0.0415, 0.9914],
    [0.0316, 0.9919, 0.0417, 0.9908],
    [0.0293, 0.9928, 0.0371, 0.9915],
    [0.0310, 0.9923, 0.0379, 0.9910],
    [0.0304, 0.9923, 0.0412, 0.9925],
    [0.0288, 0.9928, 0.0576, 0.9890],
    [0.0282, 0.9934, 0.0421, 0.9925],
    [0.0286, 0.9924, 0.0522, 0.9902],
    [0.0301, 0.9930, 0.0511, 0.9898],
    [0.0266, 0.9932, 0.0459, 0.9897],
    [0.0243, 0.9940, 0.0494, 0.9925],
    [0.0275, 0.9934, 0.0537, 0.9903],
    [0.0261, 0.9934, 0.0435, 0.9930],
    [0.0264, 0.9938, 0.0483, 0.9909],
    [0.0216, 0.9947, 0.0526, 0.9912],
    [0.0260, 0.9938, 0.0441, 0.9918],
    [0.0267, 0.9937, 0.0476, 0.9918],
    [0.0244, 0.9942, 0.0559, 0.9904],
    [0.0281, 0.9936, 0.0619, 0.9909],
    [0.0269, 0.9935, 0.0666, 0.9913],
    [0.0347, 0.9920, 0.0548, 0.9881],
    [0.0318, 0.9928, 0.0556, 0.9906],
    [0.0248, 0.9943, 0.0445, 0.9912],
    [0.0261, 0.9936, 0.0579, 0.9924],
    [0.0221, 0.9950, 0.0581, 0.9914],
    [0.0188, 0.9957, 0.0693, 0.9912],
    [0.0216, 0.9948, 0.0700, 0.9915],
    [0.0309, 0.9932, 0.0572, 0.9906],
    [0.0234, 0.9942, 0.0718, 0.9917],
    [0.0320, 0.9930, 0.0601, 0.9908],
    [0.0235, 0.9952, 0.0551, 0.9920],
    [0.0237, 0.9946, 0.0818, 0.9904],
    [0.0361, 0.9921, 0.0654, 0.9923],
    [0.0191, 0.9953, 0.0763, 0.9923],
    [0.0296, 0.9942, 0.0631, 0.9925],
    [0.0198, 0.9956, 0.0727, 0.9925],
    [0.0307, 0.9933, 0.0737, 0.9914],
    [0.0218, 0.9949, 0.0772, 0.9921]
])

np.diff(arr, axis=0)

array([[-3.537e-01,  1.163e-01, -2.180e-02,  5.100e-03],
       [-3.100e-02,  8.900e-03, -7.500e-03,  2.300e-03],
       [-1.320e-02,  1.800e-03,  8.200e-03, -1.200e-03],
       [-7.900e-03,  2.800e-03, -1.910e-02,  3.900e-03],
       [-7.700e-03,  1.600e-03,  1.330e-02, -2.000e-04],
       [ 3.000e-04, -3.000e-04, -5.600e-03, -6.000e-04],
       [-5.400e-03,  1.900e-03, -3.000e-03,  8.000e-04],
       [ 1.000e-04, -3.000e-04,  4.400e-03,  7.000e-04],
       [-1.300e-03,  1.000e-04,  2.230e-02, -5.500e-03],
       [ 1.000e-03,  1.000e-04, -1.650e-02,  4.800e-03],
       [-9.700e-03,  2.300e-03,  1.140e-02, -1.300e-03],
       [ 4.300e-03, -1.200e-03, -1.760e-02,  1.900e-03],
       [-2.500e-03,  7.000e-04,  2.000e-04, -6.000e-04],
       [-2.300e-03,  9.000e-04, -4.600e-03,  7.000e-04],
       [ 1.700e-03, -5.000e-04,  8.000e-04, -5.000e-04],
       [-6.000e-04,  0.000e+00,  3.300e-03,  1.500e-03],
       [-1.600e-03,  5.000e-04,  1.640e-02, -3.500e-03],
       [-6.000e-04,  6.000e-04,

In [ ]:
# plt.plot(arr[1:,[1,3]])

In [11]:
# model.fit(x_train, y_train, batch_size=128, epochs=50, verbose=1,
#           validation_split=0.2, initial_epoch=10)

Train on 48000 samples, validate on 12000 samples
Epoch 11/50
48000/48000 [==============================] - 49s 1ms/step - loss: 0.0395 - accuracy: 0.9901 - val_loss: 0.0477 - val_accuracy: 0.9908
Epoch 12/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0298 - accuracy: 0.9924 - val_loss: 0.0591 - val_accuracy: 0.9895
Epoch 13/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0341 - accuracy: 0.9912 - val_loss: 0.0415 - val_accuracy: 0.9914
Epoch 14/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0316 - accuracy: 0.9919 - val_loss: 0.0417 - val_accuracy: 0.9908
Epoch 15/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0293 - accuracy: 0.9928 - val_loss: 0.0371 - val_accuracy: 0.9915
Epoch 16/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0310 - accuracy: 0.9923 - val_loss: 0.0379 - val_accuracy: 0.9910
Epoch 17/50
48000/48000 [==============================] - 51s

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, callbacks=[checkpoint],
          validation_data=(x_test, y_test), initial_epoch=10)

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=30, verbose=1, callbacks=[checkpoint],
          validation_data=(x_test, y_test), initial_epoch=20)

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=40, verbose=1, callbacks=[checkpoint],
          validation_data=(x_test, y_test), initial_epoch=30)

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=50, verbose=1, callbacks=[checkpoint],
          validation_data=(x_test, y_test), initial_epoch=40)